A Big Thanks to the notebook contributer to let me get the ideas to solve this question as a beginner!
1. [3 hooman 1 cat - Logistics](http://www.kaggle.com/xuanquang1999/3-hooman-1-cat-logistics)
2. [week5](http://www.kaggle.com/huikang/week5)

# Tips to get 1.0
There are some tips to get 1.0 from this disscussion. Another big thanks!(https://www.kaggle.com/c/open-shopee-code-league-logistic/discussion/165972):
> 1. You can easily count the weekday difference with the help of numpy.busday_count function! Check the "weekmask" and "holidays" in numpy document.
> 2. We use pd.to_datetime to process the epoch time from "deliveryordersmarch.csv". However, don't forget we need to convert the time to GMT+8.
> 3. I learned from the discussion forum [1] that if the weekday of "pick" is Sunday, we need to shift it to next day.
> 4. You can use datetime.timedelta() to add a day or a hour to a datetime object.
> 
> [1]. https://www.kaggle.com/c/open-shopee-code-league-logistic/discussion/165829

# Set environment

In [1]:
import numpy as np
import pandas as pd

# Data manipulate

In [2]:
data = pd.read_csv('../input/logistics-shopee-code-league/delivery_orders_march.csv')

In [3]:
# look at data
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


In [5]:
# get the state of the address by getting the last str in the address
data['buyeraddress'] = data['buyeraddress'].apply(lambda x: x.split()[-1]).str.lower()
data['selleraddress'] = data['selleraddress'].apply(lambda x: x.split()[-1]).str.lower()
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,manila,manila
1,2219624609,1583309968,1.583463e+09,1.583799e+09,manila,manila
2,2220979489,1583306434,1.583460e+09,NaN,manila,manila
3,2221066352,1583419016,1.583556e+09,NaN,manila,manila
4,2222478803,1583318305,1.583480e+09,NaN,luzon,manila


In [6]:
# calculate the business days to deliver before late 
temp = []
for i,j in data[['buyeraddress','selleraddress']].itertuples(index=False):
  if (i == 'manila' and j == 'manila'):
    temp.append(3)
  elif (
      (i == 'manila' and j == 'luzon')
      or (i == 'luzon' and j == 'manila')
      or (i == 'luzon' and j == 'luzon')
      ):
    temp.append(5)
  else:
    temp.append(7)

data['days'] = temp
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days
0,2215676524,1583138397,1.583385e+09,NaN,manila,manila,3
1,2219624609,1583309968,1.583463e+09,1.583799e+09,manila,manila,3
2,2220979489,1583306434,1.583460e+09,NaN,manila,manila,3
3,2221066352,1583419016,1.583556e+09,NaN,manila,manila,3
4,2222478803,1583318305,1.583480e+09,NaN,luzon,manila,5


In [7]:
# change unix time to date
# need to convert to gmt+8
data[['pick','1st_deliver_attempt','2nd_deliver_attempt']] += 8*60*60
data['pick'] = pd.to_datetime(data['pick'],unit='s').dt.date
data['1st_deliver_attempt'] = pd.to_datetime(data['1st_deliver_attempt'],unit='s').dt.date
data['2nd_deliver_attempt'] = data['2nd_deliver_attempt'].replace(np.nan,0) # change nan to 0 or else can't be process
data['2nd_deliver_attempt'] = pd.to_datetime(data['2nd_deliver_attempt'],unit='s').dt.date
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila,3
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila,5


In [8]:
# count how many days of business day taken for the 1pick and 2pick
holiday = ['2020-03-08','2020-03-25','2020-03-30','2020-03-31']
data['1st_pick'] = np.busday_count(data['pick'], data['1st_deliver_attempt'], weekmask='1111110', holidays=holiday)
data['2nd_pick'] = np.busday_count(data['1st_deliver_attempt'], data['2nd_deliver_attempt']	, weekmask='1111110', holidays=holiday)
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days,1st_pick,2nd_pick
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila,3,3,-15708
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3,2,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila,3,2,-15709
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila,3,2,-15710
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila,5,2,-15709


In [9]:
# check if is late
data['is_late'] = (data['1st_pick'] > data['days']) | (data['2nd_pick'] > 3)
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,days,1st_pick,2nd_pick,is_late
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila,3,3,-15708,False
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3,2,3,False
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila,3,2,-15709,False
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila,3,2,-15710,False
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila,5,2,-15709,False


# Prepare Submit

In [10]:
# prepare submission df and change is_late column to int using .apply(int)
submission = pd.DataFrame({'orderid':data['orderid'], 'is_late':data['is_late'].apply(int)})
submission

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
...,...,...
3176308,31504087640510,0
3176309,31504147352227,0
3176310,31504462290482,0
3176311,31504851495943,0


In [11]:
submission.to_csv('submission.csv', header=True, index=False)